### Imports

In [1]:
import os
import sys
import yaml
import argparse
import numpy as np
import torch

from hls4ml.utils.config import config_from_pyg_model
from hls4ml.converters import convert_from_pyg_model
# module_path = os.path.abspath(os.path.join('../pyg_to_hls_hls4ml/hls4ml/utils'))
# print(module_path)
# if module_path not in sys.path:
#     sys.path.append(module_path)
# from config import config_from_pyg_model

# module_path = os.path.abspath(os.path.join('../pyg_to_hls_hls4ml/hls4ml'))
# print(module_path)
# if module_path not in sys.path:
#     sys.path.append(module_path)
# from converters import convert_from_pyg_model


from collections import OrderedDict
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

# locals
from utils.models.interaction_network_pyg import InteractionNetwork
from model_wrappers import model_wrapper
from utils.data.dataset_pyg import GraphDataset
from utils.data.fix_graph_size import fix_graph_size

layer: Conv1d
func: <function parse_conv1d_layer at 0x7f8d09b567a0>
layer: Conv2d
func: <function parse_conv2d_layer at 0x7f8d09b56830>
layer: Linear
func: <function parse_linear_layer at 0x7f8d09b569e0>
layer: Softmax
func: <function parse_activation_layer at 0x7f8d09b56950>
layer: ReLU
func: <function parse_activation_layer at 0x7f8d09b56950>
layer: BatchNorm2d
func: <function parse_batchnorm_layer at 0x7f8d09b568c0>
layer: BatchNorm1d
func: <function parse_batchnorm_layer at 0x7f8d09b568c0>
handler args: ('NodeBlock',)
handler args: ('EdgeBlock',)
handler args: ('EdgeAggregate',)
handler args: ('ResidualBlock',)
handler args: ('NodeEncoder',)
handler args: ('EdgeEncoder',)
layer: NodeBlock
func: <function parse_NodeBlock at 0x7f8d09b6a050>
layer: EdgeBlock
func: <function parse_EdgeBlock at 0x7f8d09b6a200>
layer: EdgeAggregate
func: <function parse_EdgeAggregate at 0x7f8d09b6a3b0>
layer: ResidualBlock
func: <function parse_ResidualBlock at 0x7f8d09b6a560>
layer: NodeEncoder
func: <f

### PyTorch Model

In [2]:
torch_model = InteractionNetwork(aggr="add", flow="source_to_target", hidden_size=40)
# torch_model_dict = torch.load("trained_models//IN_pyg_small_add_source_to_target_40_state_dict.pt")
# torch_model.load_state_dict(torch_model_dict)

for name, submodule in torch_model.named_modules():
    if name != "":
        print(f"{name}: {submodule}")

R1: RelationalModel(
  (layers): Sequential(
    (0): Linear(in_features=15, out_features=40, bias=True)
    (1): ReLU()
    (2): Linear(in_features=40, out_features=40, bias=True)
    (3): ReLU()
    (4): Linear(in_features=40, out_features=5, bias=True)
  )
)
R1.layers: Sequential(
  (0): Linear(in_features=15, out_features=40, bias=True)
  (1): ReLU()
  (2): Linear(in_features=40, out_features=40, bias=True)
  (3): ReLU()
  (4): Linear(in_features=40, out_features=5, bias=True)
)
R1.layers.0: Linear(in_features=15, out_features=40, bias=True)
R1.layers.1: ReLU()
R1.layers.2: Linear(in_features=40, out_features=40, bias=True)
R1.layers.3: ReLU()
R1.layers.4: Linear(in_features=40, out_features=5, bias=True)
O: ObjectModel(
  (layers): Sequential(
    (0): Linear(in_features=5, out_features=40, bias=True)
    (1): ReLU()
    (2): Linear(in_features=40, out_features=40, bias=True)
    (3): ReLU()
    (4): Linear(in_features=40, out_features=5, bias=True)
  )
)
O.layers: Sequential(
  (

### HLS Model

In [3]:
# forward_dict: defines the order in which graph-blocks are called in the model's 'forward()' method
forward_dict = OrderedDict()
forward_dict["node_encoder"] = "NodeEncoder"
forward_dict["edge_encoder"] = "EdgeEncoder"
forward_dict["R1"] = "EdgeBlock"
forward_dict["O"] = "NodeBlock"
forward_dict["res_block"] = "ResidualBlock"
forward_dict["R2"] = "EdgeBlock"

In [4]:
common_dim = 5
graph_dims = {
        "n_node": 28,
        "n_edge": 37,
#         "node_dim": 3,
        "node_attr": 3,
        "node_dim": common_dim,
#         "edge_dim": 4,
        "edge_attr": 4,
    "edge_dim":common_dim
}

In [5]:
# output_dir = "test_GNN"
# config = config_from_pyg_model(torch_model,
#                                    default_precision="ap_fixed<32,12>",
#                                    default_index_precision='ap_uint<16>', 
#                                    default_reuse_factor=1)
# hls_model = convert_from_pyg_model(torch_model,
#                                        n_edge=graph_dims['n_edge'],
#                                        n_node=graph_dims['n_node'],
#                                        edge_attr=graph_dims['edge_attr'],
#                                        node_attr=graph_dims['node_attr'],
#                                        edge_dim=graph_dims['edge_dim'],
#                                        node_dim=graph_dims['node_dim'],
#                                        forward_dictionary=forward_dict, 
#                                        activate_final='sigmoid',
#                                        output_dir=output_dir,
#                                        hls_config=config)

In [6]:
output_dir = "test_GNN"
config = config_from_pyg_model(torch_model,
                                   default_precision="ap_fixed<16,8>",
                                   default_index_precision='ap_uint<16>', 
                                   default_reuse_factor=8)
print(f"config: {config}")
hls_model = convert_from_pyg_model(torch_model,
                                       n_edge=graph_dims['n_edge'],
                                       n_node=graph_dims['n_node'],
                                       edge_attr=graph_dims['edge_attr'],
                                       node_attr=graph_dims['node_attr'],
                                       edge_dim=graph_dims['edge_dim'],
                                       node_dim=graph_dims['node_dim'],
                                       forward_dictionary=forward_dict, 
                                       activate_final='sigmoid',
                                       output_dir=output_dir,
                                       hls_config=config)

config: {'Model': {'Precision': 'ap_fixed<16,8>', 'IndexPrecision': 'ap_uint<16>', 'ReuseFactor': 8, 'Strategy': 'Latency'}}
PygModelReader node_dim: 5
PygModelReader edge_dim: 5
PygModelReader node_attr: 3
PygModelReader edge_attr: 4
node encoder layer_dict['n_in']: 3
node encoder layer_dict['n_out']: 5
edge encoder b4 update_dict["last_edge_update"]: edge_attr
edge encoder after update_dict["last_edge_update"]: layer5_out
EdgeBlock update_dict["last_edge_update"] b4: layer5_out
EdgeBlock update_dict["last_edge_update"] after: layer6_out
type(update_dict): <class 'dict'>
nodeblock layer_dict: {'name': 'O', 'n_node': 28, 'n_edge': 37, 'node_dim': 5, 'edge_dim': 5, 'activate_final': 'false', 'activation': 'linear', 'n_layers': 3, 'out_dim': 5, 'class_name': 'NodeBlock', 'inputs': ['layer4_out', 'layer7_out'], 'outputs': ['layer8_out']}
layer_dict['inputs']: ['layer4_out', 'layer8_out']
EdgeBlock update_dict["last_edge_update"] b4: layer6_out
EdgeBlock update_dict["last_edge_update"] aft

In [7]:
hls_model.compile()

Writing HLS project
layer.get_variables(): odict_values([<hls4ml.model.hls_layers.ArrayVariable object at 0x7f8cd69b7ad0>])
layer name: node_attr. func: None
layer.get_variables(): odict_values([<hls4ml.model.hls_layers.ArrayVariable object at 0x7f8cd69b7cd0>])
layer name: edge_attr. func: None
layer.get_variables(): odict_values([<hls4ml.model.hls_layers.ArrayVariable object at 0x7f8cd6999f90>])
layer name: edge_index. func: None
layer.get_variables(): odict_values([<hls4ml.model.hls_layers.ArrayVariable object at 0x7f8ee430c3d0>])
def_cpp: layer4_t layer4_out[N_LAYER_1_4*N_LAYER_2_4]
layer name: node_encoder. func: ['nnet::encoder<input_t, layer4_t, config4>(node_attr, layer4_out, w4, b4);']
layer.get_variables(): odict_values([<hls4ml.model.hls_layers.ArrayVariable object at 0x7f8cf008e450>])
def_cpp: layer5_t layer5_out[N_LAYER_1_5*N_LAYER_2_5]
layer name: edge_encoder. func: ['nnet::encoder<input2_t, layer5_t, config5>(edge_attr, layer5_out, w5, b5);']
layer.get_variables(): odict

Done
lib_name: firmware/myproject-E26CcE0d.so


# Evaluation and prediction: hls_model.predict(input)

### Data

In [8]:
class data_wrapper(object):
    def __init__(self, node_attr, edge_attr, edge_index, target):
        self.x = node_attr
        self.edge_attr = edge_attr
        self.edge_index = edge_index.transpose(0,1)

        node_attr, edge_attr, edge_index = self.x.detach().cpu().numpy(), self.edge_attr.detach().cpu().numpy(), self.edge_index.transpose(0, 1).detach().cpu().numpy().astype(np.float32)
        node_attr, edge_attr, edge_index = np.ascontiguousarray(node_attr), np.ascontiguousarray(edge_attr), np.ascontiguousarray(edge_index)
        self.hls_data = [node_attr, edge_attr, edge_index]

        self.target = target
        self.np_target = np.reshape(target.detach().cpu().numpy(), newshape=(target.shape[0],))

def load_graphs(graph_indir, graph_dims, n_graphs):
    graph_files = np.array(os.listdir(graph_indir))
    graph_files = np.array([os.path.join(graph_indir, graph_file)
                            for graph_file in graph_files])
    n_graphs_total = len(graph_files)
    IDs = np.arange(n_graphs_total)
    print(f"IDS: {IDs}")
    dataset = GraphDataset(graph_files=graph_files[IDs])

    graphs = []
    for data in dataset[:n_graphs]:
        node_attr, edge_attr, edge_index, target, bad_graph = fix_graph_size(data.x, data.edge_attr, data.edge_index,
                                                                             data.y,
                                                                             n_node_max=graph_dims['n_node'],
                                                                             n_edge_max=graph_dims['n_edge'])
#         if not bad_graph:
#             graphs.append(data_wrapper(node_attr, edge_attr, edge_index, target))
        graphs.append(data_wrapper(node_attr, edge_attr, edge_index, target))
    print(f"graphs length: {len(graphs)}")

    print("writing test bench data for 1st graph")
    data = graphs[0]
    node_attr, edge_attr, edge_index = data.x.detach().cpu().numpy(), data.edge_attr.detach().cpu().numpy(), data.edge_index.transpose(
        0, 1).detach().cpu().numpy().astype(np.int32)
    os.makedirs('tb_data', exist_ok=True)
    input_data = np.concatenate([node_attr.reshape(1, -1), edge_attr.reshape(1, -1), edge_index.reshape(1, -1)], axis=1)
    np.savetxt('tb_data/input_data.dat', input_data, fmt='%f', delimiter=' ')

    return graphs


graph_indir = "trackml_data/processed_plus_pyg_small"
# graph_dims = {
#         "n_node": 28,
#         "n_edge": 37,
#         "node_dim": 3,
#         "edge_dim": 4
#     }
graphs = load_graphs(graph_indir, graph_dims, n_graphs=10)

IDS: [0 1 2 3 4 5 6 7 8 9]
graphs length: 10
writing test bench data for 1st graph


In [9]:
data = graphs[0]
print(f"graphs len: {len(graphs)}")
print(type(data.hls_data))
print(f"data.x shape:{data.x.shape}")
torch_pred = torch_model(data)
torch_pred = torch_pred.detach().cpu().numpy().flatten()
print(f"torch pred shape: {torch_pred.shape}")
hls_pred = hls_model.predict(data.hls_data)
print(f"hls_pred.shape: {hls_pred.shape}")
MSE = mean_squared_error(torch_pred, hls_pred)
print(f"MSE: {MSE}")

print((np.abs(torch_pred- hls_pred)))
print(torch_pred)
print(hls_pred)

graphs len: 10
<class 'list'>
data.x shape:torch.Size([28, 3])
Node Encoder output mean: -0.1947786808013916. std: 0.3186385929584503
Edge Encoder output mean: 0.07116236537694931. std:0.21981175243854523
R1 output mean: -0.0005930254119448364, std: 0.14667610824108124
O output mean: -0.09151472896337509, std: 0.09432750195264816
R2 output mean: 0.17527851462364197, std: 0.0140219172462821
torch pred shape: (37,)
hls_pred.shape: (37,)
MSE: 0.0003597439790610224
[0.02195287 0.0181613  0.02208006 0.01828885 0.01103634 0.01115906
 0.01109821 0.01122272 0.01056659 0.01065809 0.01447999 0.01066762
 0.02612233 0.02608162 0.02609205 0.02605367 0.02549201 0.01652002
 0.01646763 0.01647258 0.01642591 0.0220626  0.020392   0.02033889
 0.0203467  0.0202983  0.02226168 0.02023661 0.02016956 0.02020359
 0.02013987 0.02238381 0.01624513 0.01617903 0.01621103 0.01614803
 0.01889557]
[0.54539037 0.54550505 0.54551756 0.5456326  0.5383801  0.5385028
 0.53844196 0.53856647 0.53791034 0.53800184 0.537917

In [10]:
def get_meanNstd(data, torch_pred, hls_pred):
#     print(data.x)
    node_mean = torch.mean(data.x)
    print(f"node_mean: {node_mean}")
    node_std = torch.std(data.x)
    print(f"node_std: {node_std}")
    node_abs_max = torch.max(torch.abs(data.x))
    print(f"node_abs_max: {node_abs_max}")
    edge_mean = torch.mean(data.edge_attr)
    print(f"edge_mean: {edge_mean}")
    edge_std = torch.std(data.edge_attr)
    print(f"edge_std: {edge_std}")
    edge_abs_max = torch.max(torch.abs(data.edge_attr))
    print(f"edge_abs_max: {edge_abs_max}")
    torch_pred_mean = np.mean(torch_pred)
    print(f"torch_pred_mean: {torch_pred_mean}")
    torch_pred_std = np.std(torch_pred)
    print(f"torch_pred_std: {torch_pred_std}")
    torch_pred_abs_max = np.max(np.abs(torch_pred))
    print(f"torch_pred_abs_max: {torch_pred_abs_max}")
    hls_pred_mean = np.mean(hls_pred)
    print(f"hls_pred_mean: {hls_pred_mean}")
    hls_pred_std = np.std(hls_pred)
    print(f"hls_pred_std: {hls_pred_std}")
    hls_pred_abs_max = np.max(np.abs(hls_pred))
    print(f"hls_pred_abs_max: {hls_pred_abs_max}")
    diff_mean = np.mean(torch_pred- hls_pred)
    print(f"diff_mean: {diff_mean}")
    diff_std = np.std(torch_pred- hls_pred)
    print(f"diff_std: {diff_std}")
    diff_abs_max = np.max(np.abs(torch_pred- hls_pred))
    print(f"diff_abs_max: {diff_abs_max}")
    return None

In [11]:
MSE_l = []
for data in graphs:
#     print(f"graphs len: {len(graphs)}")
#     print(type(data.hls_data))
#     print(f"data.x shape:{data.x.shape}")
    torch_pred = torch_model(data)
    torch_pred = torch_pred.detach().cpu().numpy().flatten()
#     print(f"torch pred shape: {torch_pred.shape}")
    hls_pred = hls_model.predict(data.hls_data)
#     print(f"hls_pred.shape: {hls_pred.shape}")
    MSE = mean_squared_error(torch_pred, hls_pred)
#     get_meanNstd(data, torch_pred, hls_pred)
    print(f"MSE: {MSE}")
    MSE_l.append(MSE)
#     MAPE = mean_absolute_percentage_error(torch_pred, hls_pred)
#     print(f"MAPE: {MAPE}")

MSE_l = np.array(MSE_l)
print(f"MSE_l: {MSE_l}")
print(f"overall MSE: {np.mean(MSE_l)}")
print(MSE_l.shape)

Node Encoder output mean: -0.1947786808013916. std: 0.3186385929584503
Edge Encoder output mean: 0.07116236537694931. std:0.21981175243854523
R1 output mean: -0.0005930254119448364, std: 0.14667610824108124
O output mean: -0.09151472896337509, std: 0.09432750195264816
R2 output mean: 0.17527851462364197, std: 0.0140219172462821
MSE: 0.0003597439790610224
Node Encoder output mean: -0.19413770735263824. std: 0.29810258746147156
Edge Encoder output mean: 0.055182863026857376. std:0.28520306944847107
R1 output mean: 0.012724081054329872, std: 0.1423618197441101
O output mean: -0.11229344457387924, std: 0.09624727815389633
R2 output mean: 0.18505193293094635, std: 0.013339392840862274
MSE: 0.0003608842089306563
Node Encoder output mean: -0.20931506156921387. std: 0.3730930984020233
Edge Encoder output mean: 0.05258729308843613. std:0.26491641998291016
R1 output mean: 0.018628841266036034, std: 0.1445307582616806
O output mean: -0.11401722580194473, std: 0.07873959839344025
R2 output mean: 0

In [12]:
for data_instance in data.hls_data:
    print(data_instance.shape)
    
print(data.edge_index.shape)

(28, 3)
(37, 4)
(37, 2)
torch.Size([2, 37])


In [13]:
# class data_wrapper_tau3mu:
#     def __init__(self, node_attr, edge_attr, edge_index, target):
#         self.x = node_attr
#         self.edge_attr = edge_attr
#         self.edge_index = edge_index.transpose(0,1)

#         node_attr, edge_attr, edge_index = self.x.detach().cpu().numpy(), self.edge_attr.detach().cpu().numpy(), self.edge_index.transpose(0, 1).detach().cpu().numpy().astype(np.float32)
#         node_attr, edge_attr, edge_index = np.ascontiguousarray(node_attr), np.ascontiguousarray(edge_attr), np.ascontiguousarray(edge_index)
#         self.hls_data = [node_attr, edge_attr, edge_index]

#         self.target = target
#         self.np_target = np.reshape(target.detach().cpu().numpy(), newshape=(target.shape[0],))

def load_graphs_tau3mu(data_loader, graph_dims:dict, n_graphs =1000):
    """
    params:
    dataloader: pyg dataloader with custom Tau3MuDataset as its dataset
    graph_dims: 
        graph_dims.keys = ["n_node": max number of nodes allowed in a graph/batch,
            "n_edge": max number of edges allowed in a graph/batch,
            "node_dim": feature dim of node,
            "edge_dim": feature dim of edge
        ]
    
    """

    graphs = []
    
    i = 0
    for data in data_loader:
        if i >= n_graphs:
            break
        data.edge_index = data.edge_index.t()#transpose the edge_index
        n_edges = data.edge_attr.shape[0]
#         print((data.y.shape))
#         print(n_edges)
        data.y = data.y.expand(1, n_edges).flatten()
#         print(f"data.y.shape: {data.y.shape}")
        node_attr, edge_attr, edge_index, target, bad_graph = fix_graph_size(data.x, 
                                                                             data.edge_attr, 
                                                                             data.edge_index,
                                                                             data.y,
                                                                             n_node_max=graph_dims['n_node'],
                                                                             n_edge_max=graph_dims['n_edge'])
        target = torch.flatten(target)# flatten target to 1d array
        if not bad_graph:
            graphs.append(data_wrapper(node_attr, edge_attr, edge_index, target))
        i +=1
        
    print(f"n_graphs: {len(graphs)}")

    print("writing test bench data for 1st graph")
    data = graphs[0]
    node_attr, edge_attr, edge_index = data.x.detach().cpu().numpy(), data.edge_attr.detach().cpu().numpy(), data.edge_index.transpose(
        0, 1).detach().cpu().numpy().astype(np.int32)
    os.makedirs('tb_data', exist_ok=True)
    input_data = np.concatenate([node_attr.reshape(1, -1), edge_attr.reshape(1, -1), edge_index.reshape(1, -1)], axis=1)
    np.savetxt('tb_data/input_data.dat', input_data, fmt='%f', delimiter=' ')

    return graphs


graph_indir = "trackml_data/processed_plus_pyg_small"
# graph_indir = "/home/swissman777/projects/Tau3MuGNNs/data"
graph_dims = {
        "n_node": 28,
        "n_edge": 37,
        "node_dim": 3,
        "edge_dim": 4
    }


In [14]:
import pickle as pkl
from torch_geometric.loader.dataloader import DataLoader
from utils.dataset import Tau3MuDataset

# with open('./trackml_data/data_loaders.pickle', 'rb') as file:
with open('./trackml_data/data_loaders_batch_size_1.pickle', 'rb') as file:
    data_loaders= pkl.load(file) #, x_dim, edge_attr_dim 

print(data_loaders)
# for stage in data_loaders.keys():
#     for data in data_loaders[stage]:
#         print(data.x.shape)
#         print(data.edge_index.shape)
#         print(data.edge_attr.shape)

# print(graphs)

{'train': <torch_geometric.loader.dataloader.DataLoader object at 0x7f8cd68deb90>, 'valid': <torch_geometric.loader.dataloader.DataLoader object at 0x7f8cd693f850>, 'test': <torch_geometric.loader.dataloader.DataLoader object at 0x7f8cd693f710>}


In [15]:
from sklearn.preprocessing import StandardScaler
MSE_l = []
stages = ["train", "valid", "test"]
node_scaler = StandardScaler()
edge_scaler = StandardScaler()
for stage in stages:
    graphs = load_graphs_tau3mu(data_loaders[stage], graph_dims)
#     print(type(graphs))

    for data in graphs: # fit data to scaler b4 feedforwarding
#         print(f"data type: {type(data)}")
        node_scaler.fit(data.x) 
        edge_scaler.fit(data.edge_attr) 
    
    for data in graphs:
       
        data.x = torch.from_numpy(node_scaler.transform(data.x)).float()
        data.edge_attr = torch.from_numpy(edge_scaler.transform(data.edge_attr)).float()
#         data.hls_data = [data.x, edge_attr, edge_index]
        data.hls_data[0] = node_scaler.transform(data.hls_data[0])
        data.hls_data[1] = edge_scaler.transform(data.hls_data[1])
        
        torch_pred = torch_model(data)
        torch_pred = torch_pred.detach().cpu().numpy()
        hls_pred = hls_model.predict(data.hls_data)
        MSE = mean_squared_error(torch_pred, hls_pred)
        print(f"MSE: {MSE}")
    #     get_meanNstd(data,torch_pred, hls_pred)
    #     print(f"MSE: {MSE}")
        MSE_l.append(MSE)
    #     MAPE = mean_absolute_percentage_error(torch_pred, hls_pred)
    #     print(f"MAPE: {MAPE}")
MSE_l = np.array(MSE_l)
print(f"MSE_l :{MSE_l}")
print(f"MSE means: {np.mean(MSE_l)}")

n_graphs: 1000
writing test bench data for 1st graph
Node Encoder output mean: -0.18221309781074524. std: 0.7065412402153015
Edge Encoder output mean: 0.13017743825912476. std:1.3330585956573486
R1 output mean: 0.04436597600579262, std: 0.1550568789243698
O output mean: -0.09579402953386307, std: 0.10822714865207672
R2 output mean: 0.24175229668617249, std: 0.0952058956027031
MSE: 0.0012110222596675158
Node Encoder output mean: -0.18438784778118134. std: 0.8286362290382385
Edge Encoder output mean: 0.13017743825912476. std:1.3639570474624634
R1 output mean: 0.04670614376664162, std: 0.16907252371311188
O output mean: -0.1079886183142662, std: 0.1208738386631012
R2 output mean: 0.22366484999656677, std: 0.07706474512815475
MSE: 0.0008239809540100396
Node Encoder output mean: -0.19613906741142273. std: 0.8791705965995789
Edge Encoder output mean: -0.2629797160625458. std:1.0913840532302856
R1 output mean: 0.06772361695766449, std: 0.19221092760562897
O output mean: -0.1181962862610817, s

MSE: 0.0009316182695329189
Node Encoder output mean: -0.17633496224880219. std: 0.9141571521759033
Edge Encoder output mean: 0.14516523480415344. std:0.47148555517196655
R1 output mean: 0.04110097140073776, std: 0.13810673356056213
O output mean: -0.07863768935203552, std: 0.1146986111998558
R2 output mean: 0.3627389371395111, std: 0.05478683486580849
MSE: 0.0026508786249905825
Node Encoder output mean: -0.18285854160785675. std: 0.7285119295120239
Edge Encoder output mean: 0.13017743825912476. std:1.2486827373504639
R1 output mean: 0.054732535034418106, std: 0.1545863151550293
O output mean: -0.09473535418510437, std: 0.10943295806646347
R2 output mean: 0.2609504759311676, std: 0.09746140986680984
MSE: 0.001497082062996924
Node Encoder output mean: -0.18402566015720367. std: 0.6699963212013245
Edge Encoder output mean: 0.13017743825912476. std:1.0100351572036743
R1 output mean: 0.02924516797065735, std: 0.15004079043865204
O output mean: -0.08294457197189331, std: 0.10859920084476471


MSE: 0.0005193391116335988
Node Encoder output mean: -0.17549222707748413. std: 1.1223409175872803
Edge Encoder output mean: 0.039395522326231. std:0.7906911969184875
R1 output mean: 0.0413789376616478, std: 0.1636723279953003
O output mean: -0.11536619812250137, std: 0.11271335929632187
R2 output mean: 0.2717055678367615, std: 0.07839508354663849
MSE: 0.001128053292632103
Node Encoder output mean: -0.17427411675453186. std: 0.8585129380226135
Edge Encoder output mean: 0.26034703850746155. std:1.361150860786438
R1 output mean: 0.05156666040420532, std: 0.1493462473154068
O output mean: -0.08731004595756531, std: 0.1080353707075119
R2 output mean: 0.2728201746940613, std: 0.08983921259641647
MSE: 0.001375026535242796
Node Encoder output mean: -0.1837591677904129. std: 0.633720874786377
Edge Encoder output mean: 0.13017743825912476. std:1.136564016342163
R1 output mean: 0.03452761471271515, std: 0.15539918839931488
O output mean: -0.0983654037117958, std: 0.10805889964103699
R2 output me

MSE: 0.001084254588931799
Node Encoder output mean: -0.18204066157341003. std: 0.8333421349525452
Edge Encoder output mean: 0.4538111388683319. std:1.4515864849090576
R1 output mean: 0.044038865715265274, std: 0.14301533997058868
O output mean: -0.0878506749868393, std: 0.10384822636842728
R2 output mean: 0.2900572717189789, std: 0.0863984227180481
MSE: 0.0015023580053821206
Node Encoder output mean: -0.19386950135231018. std: 0.8916423916816711
Edge Encoder output mean: 0.007919630967080593. std:0.8534330129623413
R1 output mean: 0.04939107596874237, std: 0.16638077795505524
O output mean: -0.11579551547765732, std: 0.11031302809715271
R2 output mean: 0.2573224604129791, std: 0.07558569312095642
MSE: 0.0010241542477160692
Node Encoder output mean: -0.1810738444328308. std: 0.5621909499168396
Edge Encoder output mean: 0.13017743825912476. std:0.8158907890319824
R1 output mean: 0.02107822336256504, std: 0.13896143436431885
O output mean: -0.0828709527850151, std: 0.11355367302894592
R2 

O output mean: -0.11239229887723923, std: 0.10892122983932495
R2 output mean: 0.21862931549549103, std: 0.07232526689767838
MSE: 0.0008224250632338226
Node Encoder output mean: -0.19533102214336395. std: 0.8224493265151978
Edge Encoder output mean: 0.10652017593383789. std:0.3871329724788666
R1 output mean: 0.04191279411315918, std: 0.16035643219947815
O output mean: -0.1211390346288681, std: 0.1132735162973404
R2 output mean: 0.1961526721715927, std: 0.04845394566655159
MSE: 0.00033575977431610227
Node Encoder output mean: -0.18923351168632507. std: 0.6863405108451843
Edge Encoder output mean: 0.13017743825912476. std:1.3167533874511719
R1 output mean: 0.0447205975651741, std: 0.16473039984703064
O output mean: -0.10444731265306473, std: 0.1085517555475235
R2 output mean: 0.2183045595884323, std: 0.07774306833744049
MSE: 0.0008040275424718857
Node Encoder output mean: -0.1799534410238266. std: 1.1464415788650513
Edge Encoder output mean: 0.14086511731147766. std:0.5561124682426453
R1 

...

MSE: 0.000605615321546793
Node Encoder output mean: -0.2119017243385315. std: 0.9572626948356628
Edge Encoder output mean: -0.004511326551437378. std:0.5110609531402588
R1 output mean: 0.05178408697247505, std: 0.17821016907691956
O output mean: -0.1424906849861145, std: 0.11436349898576736
R2 output mean: 0.22242285311222076, std: 0.04148413985967636
MSE: 0.0006699501536786556
Node Encoder output mean: -0.20963789522647858. std: 0.6851147413253784
Edge Encoder output mean: 0.09037812054157257. std:2.5113160610198975
R1 output mean: 0.04968145117163658, std: 0.20763754844665527
O output mean: -0.13006852567195892, std: 0.11397092044353485
R2 output mean: 0.2161279171705246, std: 0.05895068123936653
MSE: 0.0008507380844093859
Node Encoder output mean: -0.20806768536567688. std: 0.5471302270889282
Edge Encoder output mean: 0.04798997566103935. std:1.9916683435440063
R1 output mean: 0.036689624190330505, std: 0.1827952265739441
O output mean: -0.12371168285608292, std: 0.1001918762922287


MSE: 0.0008770653512328863
Node Encoder output mean: -0.21174997091293335. std: 0.8060033917427063
Edge Encoder output mean: -0.19943346083164215. std:1.2114605903625488
R1 output mean: 0.05584493651986122, std: 0.21323195099830627
O output mean: -0.14059580862522125, std: 0.11449214071035385
R2 output mean: 0.23983249068260193, std: 0.03320026770234108
MSE: 0.0008856559288688004
Node Encoder output mean: -0.2020682692527771. std: 0.6794537305831909
Edge Encoder output mean: -0.011724961921572685. std:0.3703120946884155
R1 output mean: 0.017951445654034615, std: 0.12926001846790314
O output mean: -0.11006072163581848, std: 0.09882713854312897
R2 output mean: 0.25479379296302795, std: 0.03582121431827545
MSE: 0.001272580586373806
Node Encoder output mean: -0.20628340542316437. std: 0.7311839461326599
Edge Encoder output mean: 0.22313711047172546. std:1.8930869102478027
R1 output mean: 0.041123926639556885, std: 0.1764044612646103
O output mean: -0.12995564937591553, std: 0.0975222438573

MSE: 0.0007324139005504549
Node Encoder output mean: -0.19944651424884796. std: 0.553883969783783
Edge Encoder output mean: 0.3019539713859558. std:1.4257293939590454
R1 output mean: 0.027260378003120422, std: 0.129068985581398
O output mean: -0.1145261600613594, std: 0.08514580875635147
R2 output mean: 0.22025778889656067, std: 0.03548811376094818
MSE: 0.0008356632897630334
Node Encoder output mean: -0.2166164517402649. std: 1.0502121448516846
Edge Encoder output mean: -0.07568766921758652. std:1.2361152172088623
R1 output mean: 0.05968714877963066, std: 0.2090621143579483
O output mean: -0.16829174757003784, std: 0.10750404745340347
R2 output mean: 0.22643247246742249, std: 0.025623494759202003
MSE: 0.0006553536513820291
Node Encoder output mean: -0.2197357416152954. std: 0.8651877641677856
Edge Encoder output mean: -0.0038763536140322685. std:2.0309665203094482
R1 output mean: 0.06752607226371765, std: 0.22598479688167572
O output mean: -0.15164946019649506, std: 0.10402143746614456

MSE: 0.000982220284640789
Node Encoder output mean: -0.22082793712615967. std: 0.9252435564994812
Edge Encoder output mean: -0.2712777256965637. std:1.4918279647827148
R1 output mean: 0.06245661526918411, std: 0.22992181777954102
O output mean: -0.1499125063419342, std: 0.11737964302301407
R2 output mean: 0.24783216416835785, std: 0.031682588160037994
MSE: 0.0009616563329473138
Node Encoder output mean: -0.21104875206947327. std: 0.6818009614944458
Edge Encoder output mean: 0.07555531710386276. std:2.2988853454589844
R1 output mean: 0.04077553749084473, std: 0.19944527745246887
O output mean: -0.13139669597148895, std: 0.1039094626903534
R2 output mean: 0.22116081416606903, std: 0.0486716628074646
MSE: 0.000788379751611501
Node Encoder output mean: -0.21290144324302673. std: 0.5840008854866028
Edge Encoder output mean: 0.047989942133426666. std:1.8040075302124023
R1 output mean: 0.038489945232868195, std: 0.19222280383110046
O output mean: -0.12681175768375397, std: 0.1093326061964035


Node Encoder output mean: -0.20649874210357666. std: 0.764631986618042
Edge Encoder output mean: -0.12572167813777924. std:1.423471212387085
R1 output mean: 0.04153851419687271, std: 0.19904011487960815
O output mean: -0.1374063491821289, std: 0.1064695343375206
R2 output mean: 0.22977624833583832, std: 0.034510038793087006
MSE: 0.0007834257557988167
Node Encoder output mean: -0.21668602526187897. std: 0.9874935150146484
Edge Encoder output mean: 0.002769561717286706. std:0.6823248267173767
R1 output mean: 0.07086963206529617, std: 0.19706614315509796
O output mean: -0.15944692492485046, std: 0.12478674948215485
R2 output mean: 0.22760120034217834, std: 0.03103436715900898
MSE: 0.0006846599280834198
Node Encoder output mean: -0.23915301263332367. std: 1.3920347690582275
Edge Encoder output mean: 0.01953861117362976. std:0.4878542423248291
R1 output mean: 0.059324171394109726, std: 0.19466657936573029
O output mean: -0.2217758148908615, std: 0.1555955857038498
R2 output mean: 0.40824565

In [16]:
MSE_l.shape

(3000,)

In [17]:
torch_model.state_dict()

OrderedDict([('R1.layers.0.weight',
              tensor([[ 1.8924e-01, -2.0537e-01, -2.0518e-01,  7.7618e-02, -4.8065e-03,
                       -9.9911e-02,  1.6971e-01,  8.6529e-02, -4.9980e-02,  2.1852e-01,
                        1.0932e-01,  8.5002e-02,  2.1813e-01,  1.0661e-01,  1.2731e-01],
                      [ 2.3636e-01,  7.9104e-02,  9.9550e-02,  2.0432e-01,  3.6054e-03,
                        1.6698e-01,  6.2341e-02, -1.6492e-01, -2.6047e-02, -2.2269e-01,
                        9.0499e-02, -1.7983e-01, -1.1080e-01, -1.4379e-02, -1.7391e-01],
                      [ 1.0170e-01,  1.3498e-01, -1.6448e-01, -2.3335e-01,  4.5892e-02,
                       -2.3777e-01,  3.0489e-02, -1.9769e-01,  2.1986e-02, -1.2673e-01,
                       -1.0220e-01, -1.0732e-01, -2.4878e-02, -2.3980e-01,  1.5108e-01],
                      [ 6.7923e-02, -5.1110e-02,  2.0601e-01,  9.9218e-02,  1.6888e-01,
                       -1.8683e-01,  1.0187e-01, -1.8784e-01,  8.3337e-02,  1.239